In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import os
os.chdir('../')

In [3]:
from src.dataset import *
from src.concept_vectors import *
from src.util import *
from src.hierarchy import *
from src.metrics import *
import numpy as np
import matplotlib.pyplot as plt

2023-01-24 19:08:41.492883: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
dataset = MNIST_Dataset()

In [6]:
seeds = [43,44,45]
hierarchy_creation_method = create_linkage_hierarchy
attributes = dataset.get_attributes()

In [10]:
metrics = [stability_metric,robustness_image_metric,responsiveness_image_metric,robustness_model_metric,responsiveness_model_metric]#truthfulness_metric]
metric_names = ['Stability', 'Image Robustness', 'Image Responsiveness','Model Robustness','Model Responsiveness']#"Truthfulness"]

### Evalaute Labels-Only vectors

In [ ]:
for metric,name in zip(metrics,metric_names):
    score = metric(hierarchy_creation_method,
                                    load_label_vectors_simple,
                                    dataset,
                                    attributes,
                                    seeds)
    print("{}: {}".format(name, score))

Stability: 0.0
Image Robustness: 0.0


### Evaluate concept2vec

In [ ]:
for metric,name in zip(metrics,metric_names):
    score = metric(hierarchy_creation_method,
                                    load_concept2vec_vectors_simple,
                                    dataset,
                                    attributes,
                                    seeds)
    print("{}: {}".format(name, score))

### Evaluate TCAV Vectors

In [12]:
for metric,name in zip(metrics,metric_names):
    score = metric(hierarchy_creation_method,
                                    load_tcav_vectors_simple,
                                    dataset,
                                    attributes,
                                    seeds)
    print("{}: {}".format(name, score))

Stability: 43.0
Image Robustness: 46.666666666666664
Image Responsiveness: 46.0
